In [1]:
from transformers import pipeline
import pandas as pd

df_2008_2010 = pd.read_csv('../data/AAPL_news_data_2008-05-18_2010-01-01.csv')
df_2010_2015 = pd.read_csv('../data/AAPL_news_data_2010-01-01_2015-01-01.csv')
df_2015_2020 = pd.read_csv('../data/AAPL_news_data_2015-01-01_2020-01-01.csv')
df_2020_2024 = pd.read_csv('../data/AAPL_news_data_2020-01-01_2024-01-01.csv')


In [2]:
# combine all the data
df_news = pd.concat([df_2008_2010, df_2010_2015, df_2015_2020, df_2020_2024], ignore_index=True)
df_news.head()
df_news.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6263 entries, 0 to 6262
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            6263 non-null   object 
 1   publish_on       6263 non-null   object 
 2   author           6263 non-null   int64  
 3   author_name      6263 non-null   object 
 4   getty_image_url  1158 non-null   object 
 5   sentiment_score  0 non-null      float64
 6   sentiment_label  0 non-null      float64
dtypes: float64(2), int64(1), object(4)
memory usage: 342.6+ KB


In [3]:
# pipe_mine = pipeline('sentiment-analysis',model='../models/distilbert-base-uncased-financial-finetune', device=0)
pipe_mine = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",device=0)     





In [4]:
result_mine = pipe_mine(df_news['title'].tolist())


In [5]:
df_mine = pd.DataFrame(result_mine)

print(df_mine.head())




      label     score
0   neutral  0.918329
1  positive  0.995474
2   neutral  0.999844
3  negative  0.998136
4   neutral  0.999879


In [6]:
df_mine['label'] = df_mine['label'].map({'negative': 0, 'neutral': 1, 'positive': 2})

In [7]:
df_news['sentiment_label'] = df_mine['label']
df_news['sentiment_score'] = df_mine['score']
df_news.head()



,title,publish_on,author,author_name,getty_image_url,sentiment_score,sentiment_label
0,FTN Midwest: Apple (NASDAQ:AAPL) remains our b...,2008-05-30T10:31:03-04:00,6383,"Eli Hoffmann, SA News Editor",NaN,0.918329,1
1,Apple's (NASDAQ:AAPL) back-to-school promo wil...,2008-05-30T08:13:35-04:00,6383,"Eli Hoffmann, SA News Editor",NaN,0.995474,2
2,New iPhone (NASDAQ:AAPL) patents reveal 1) A w...,2008-05-29T13:29:55-04:00,6383,"Eli Hoffmann, SA News Editor",NaN,0.999844,1
3,Infineon's (IFX -12.85%) warning this morning ...,2008-05-29T11:07:50-04:00,6383,"Eli Hoffmann, SA News Editor",NaN,0.998136,0
4,Hutchison Telecom (HTX) to sell iPhone (NASDAQ...,2008-05-29T10:59:38-04:00,6383,"Eli Hoffmann, SA News Editor",NaN,0.999879,1


In [8]:
# create positive_score and negative_score
df_news['positive_score'] = df_news['sentiment_score'] * df_news['sentiment_label']
df_news['negative_score'] = df_news['sentiment_score'] * (df_news['sentiment_label'] - 1)


In [9]:
df_news['Date'] = df_news['publish_on'].str[:10]
df_news['Date'] = pd.to_datetime(df_news['Date'])
df_news.drop(columns=['getty_image_url'], inplace=True)
df_news.head()

,title,publish_on,author,author_name,sentiment_score,sentiment_label,positive_score,negative_score,Date
0,FTN Midwest: Apple (NASDAQ:AAPL) remains our b...,2008-05-30T10:31:03-04:00,6383,"Eli Hoffmann, SA News Editor",0.918329,1,0.918329,0.000000,2008-05-30
1,Apple's (NASDAQ:AAPL) back-to-school promo wil...,2008-05-30T08:13:35-04:00,6383,"Eli Hoffmann, SA News Editor",0.995474,2,1.990949,0.995474,2008-05-30
2,New iPhone (NASDAQ:AAPL) patents reveal 1) A w...,2008-05-29T13:29:55-04:00,6383,"Eli Hoffmann, SA News Editor",0.999844,1,0.999844,0.000000,2008-05-29
3,Infineon's (IFX -12.85%) warning this morning ...,2008-05-29T11:07:50-04:00,6383,"Eli Hoffmann, SA News Editor",0.998136,0,0.000000,-0.998136,2008-05-29
4,Hutchison Telecom (HTX) to sell iPhone (NASDAQ...,2008-05-29T10:59:38-04:00,6383,"Eli Hoffmann, SA News Editor",0.999879,1,0.999879,0.000000,2008-05-29


In [10]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6263 entries, 0 to 6262
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   title            6263 non-null   object        
 1   publish_on       6263 non-null   object        
 2   author           6263 non-null   int64         
 3   author_name      6263 non-null   object        
 4   sentiment_score  6263 non-null   float64       
 5   sentiment_label  6263 non-null   int64         
 6   positive_score   6263 non-null   float64       
 7   negative_score   6263 non-null   float64       
 8   Date             6263 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 440.5+ KB


In [11]:
df_news.to_csv('../data/AAPL_news_data_sentiment.csv', index=False)